# Kernel density estimation

Tests on the titanic dataset.

In [10]:
import pandas as pd, numpy as np
%matplotlib inline

In [11]:
train = pd.read_csv('./datasets/titanic_train.csv')
test  = pd.read_csv('./datasets/titanic_test.csv')

### Data preparation + modeling

In [13]:
data = pd.concat([train, test])

to_drop = ['PassengerId', 'Ticket', 'Cabin', 'Name', 'Embarked', 'Parch', 'Pclass', 'Sex', 'SibSp']
data.drop(to_drop, axis=1, inplace=True)

#data.Sex, mapper_sex      = pd.factorize(data.Sex)
#data.Embarked, mapper_emb = pd.factorize(data.Embarked)

data['Age'] = data['Age'].fillna(data['Age'].median())
data['Fare'] = data['Fare'].fillna(data['Fare'].median())

In [14]:
# Normalizing
cont = ['Age', 'Fare']
cate = ['Embarked', 'Parch', 'Pclass', 'Sex', 'SibSp']

normalized = data.copy()
normalized[cont] = (normalized[cont] - normalized[cont].mean()) / normalized[cont].std()

In [15]:
train = normalized[~pd.isnull(normalized.Survived)].fillna(0.0)
test  = normalized.loc[pd.isnull(normalized.Survived)].fillna(0.0)

test.drop('Survived', axis=1, inplace=True)

In [16]:
from sklearn.ensemble import RandomForestClassifier

train_x = train.drop('Survived', axis=1)
train_y = train.Survived

rf = RandomForestClassifier(n_estimators=250)
rf.fit(train_x, train_y)

preds = rf.predict(test)

### Kernel density estimation

In [17]:
X = test.iloc[14]
X

Age     1.355791
Fare    0.539101
Name: 14, dtype: float64

In [ ]:
cont = ['Age', 'Fare']
cate = ['Embarked', 'Parch', 'Pclass', 'Sex', 'SibSp']

samples = pd.DataFrame(columns=train_x.columns)
N = 10

for col in train_x:
    if col in cont:
        samples[col] = np.random.normal(X[col], np.std(train_x[col]), N)
    elif col in cate:
        samples[col] = np.random.randint(min(train_x[col]), max(train_x[col]) + 1, N)
        
samples['Survived'] = rf.predict_proba(samples)[:,1]

In [ ]:
samples

In [ ]:
kernels = scipy.stats.gaussian_kde(samples.values.T)

kernels.evaluate(X)

In [ ]:
X.values

# Sample from unknown - then KDE

In [21]:
rf.predict_proba([[23, 150]])[0,1]

0.42799999999999999

In [24]:
# To use automatic gradient calculations, use numpy (np) provided
# by autograd through Sampyl
import sampyl as smp
from sampyl import np
import seaborn

icov = np.linalg.inv(np.array([[1., .8], [.8, 1.]]))
def logp(age, fare):
    d = np.array([age, fare])
    return rf.predict_proba([d])[0,1]

start = {'age': 1.35, 'fare': 0.53}
metro_hast = smp.Metropolis(logp, start)
chain = metro_hast.sample(1000)

seaborn.jointplot(chain.age, chain.fare, stat_func=None)

Progress: [##----------------------------] 76 of 1000 samples

KeyboardInterrupt: 